In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
import datetime

plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell


InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
# warnings.filterwarnings("ignore")

df = pd.read_csv("2018.csv", low_memory = False)
#print('Dataframe dimensions:', df.shape)
#tab_info=pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
#tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values (nb)'}))
#tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100)
                     #.T.rename(index={0:'null values (%)'}))
#print(tab_info)
df = df[df.FL_DATE.str.contains('^2018-01')]
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
#print(df.shape)
#print(df[0:10][:])
attributes_to_remove = ['OP_CARRIER_FL_NUM', 'CRS_DEP_TIME', 'DEP_TIME', 'WHEELS_OFF', 'WHEELS_ON', 
                        'CRS_ARR_TIME', 'ARR_TIME', 'CANCELLATION_CODE', 'DIVERTED','CARRIER_DELAY', 
                        'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 
                        'CRS_ELAPSED_TIME', 'AIR_TIME']
df.drop(attributes_to_remove, axis = 1, inplace = True)
# df = df[['FL_DATE', 'OP_CARRIER', 'ORIGIN', 'DEST',
#     'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_IN', 
#     'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
#     'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME']]
print(df[:5])

      FL_DATE OP_CARRIER ORIGIN DEST  DEP_DELAY  TAXI_OUT  TAXI_IN  ARR_DELAY  \
0  2018-01-01         UA    EWR  DEN       -5.0      15.0     10.0      -23.0   
1  2018-01-01         UA    LAS  SFO       -8.0      11.0      7.0      -24.0   
2  2018-01-01         UA    SNA  DEN       -5.0      15.0      5.0      -13.0   
3  2018-01-01         UA    RSW  ORD        6.0      19.0      6.0       -2.0   
4  2018-01-01         UA    ORD  ALB       20.0      13.0     10.0       14.0   

   CANCELLED  ACTUAL_ELAPSED_TIME  DISTANCE  
0        0.0                250.0    1605.0  
1        0.0                 83.0     414.0  
2        0.0                126.0     846.0  
3        0.0                182.0    1120.0  
4        0.0                106.0     723.0  


In [2]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
        'count': group.count(), 'mean': group.mean()}
#global_stats = df['TAXI_IN'].groupby(df['OP_CARRIER']).apply(get_stats).unstack()
#global_stats = global_stats.sort_values('count')
#print(global_stats)
def format_hour_minute(time):
    if pd.isnull(time):
        return np.nan
    else:
        if time == 2400: time = 0
        time = "{0:04d}".format(int(time))
        hour_minute = datetime.time(int(time[0:2]), int(time[2:4]))
        return hour_minute
def combine_date_hour_minute(x):
    if pd.isnull(x[0]) or pd.isnull(x[1]):
        return np.nan
    else:
        return datetime.datetime.combine(x[0],x[1])
def create_flight_time(df, col):    
    liste = []
    for index, cols in df[['FL_DATE', col]].iterrows():    
        if pd.isnull(cols[1]):
            liste.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days=1)
            cols[1] = datetime.time(0,0)
            liste.append(combine_date_hour_minute(cols))
        else:
            cols[1] = format_hour_minute(cols[1])
            liste.append(combine_date_hour_minute(cols))
    return pd.Series(liste)
le = LabelEncoder()

df['FL_DATE'] = le.fit_transform(df['FL_DATE'])
df['OP_CARRIER'] = le.fit_transform(df['OP_CARRIER'])
df['ORIGIN'] = le.fit_transform(df['ORIGIN'])
df['DEST'] = le.fit_transform(df['DEST'])
# df['FL_DATE'] = create_flight_time(df, 'CRS_DEP_TIME')
# df['DEP_TIME'] = df['DEP_TIME'].apply(format_hour_minute)
# df['CRS_ARR_TIME'] = df['CRS_ARR_TIME'].apply(format_hour_minute)
# df['ARR_TIME'] = df['ARR_TIME'].apply(format_hour_minute)

print(df)
df = df.dropna()
print(df['TAXI_IN'])
df['TAXI_IN'].to_pickle("./label.pkl")
df.loc[:, df.columns != 'TAXI_IN'].to_pickle('./pure_data.pkl')


        FL_DATE  OP_CARRIER  ORIGIN  DEST  DEP_DELAY  TAXI_OUT  TAXI_IN  \
0             0          13     104    85       -5.0      15.0     10.0   
1             0          13     173   288       -8.0      11.0      7.0   
2             0          13     299    85       -5.0      15.0      5.0   
3             0          13     274   230        6.0      19.0      6.0   
4             0          13     230    12       20.0      13.0     10.0   
...         ...         ...     ...   ...        ...       ...      ...   
570113       30           4      18   156       -8.0       8.0      3.0   
570114       30           4     156    18       -6.0      10.0      9.0   
570115       30           4     293   217       -8.0      22.0     16.0   
570116       30           4      18   312       -5.0      10.0      4.0   
570117       30           4     312    18       -5.0      11.0      8.0   

        ARR_DELAY  CANCELLED  ACTUAL_ELAPSED_TIME  DISTANCE  
0           -23.0        0.0         

In [3]:
# print(df_train)
# print(df_test)
# df_train.dropna().to_pickle("./train.pkl")
# df_test.dropna().to_pickle("./test.pkl")

          FL_DATE OP_CARRIER ORIGIN DEST  DEP_DELAY  TAXI_OUT  TAXI_IN  \
0      2018-01-01         UA    EWR  DEN       -5.0      15.0     10.0   
1      2018-01-01         UA    LAS  SFO       -8.0      11.0      7.0   
2      2018-01-01         UA    SNA  DEN       -5.0      15.0      5.0   
3      2018-01-01         UA    RSW  ORD        6.0      19.0      6.0   
4      2018-01-01         UA    ORD  ALB       20.0      13.0     10.0   
...           ...        ...    ...  ...        ...       ...      ...   
404694 2018-01-22         DL    ATL  IND       -2.0      13.0      6.0   
404695 2018-01-22         DL    IND  ATL       -5.0      11.0     10.0   
404696 2018-01-22         DL    SJC  MSP       55.0      29.0      NaN   
404697 2018-01-22         DL    ATL  SYR       -5.0      12.0      5.0   
404698 2018-01-22         DL    SYR  ATL       -5.0      12.0      7.0   

        ARR_DELAY  CANCELLED  CRS_ELAPSED_TIME  ACTUAL_ELAPSED_TIME  AIR_TIME  \
0           -23.0        0.0  